<a href="https://colab.research.google.com/github/jahelsantiago/tensorflow_tutorial/blob/master/cats_v_dogs_v2(transfer_learning).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
from tensorflow.keras import layers
from tensorflow.keras import Model
from tensorflow.keras.applications.inception_v3 import InceptionV3


In [2]:
#descargamos el model  inception
!wget --no-check-certificate \
    https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5 \
    -O /tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5

--2020-08-10 00:39:40--  https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5
Resolving storage.googleapis.com (storage.googleapis.com)... 108.177.125.128, 74.125.203.128, 64.233.189.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|108.177.125.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 87910968 (84M) [application/x-hdf]
Saving to: ‘/tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5’

/tmp/inception_v3_w 100%[===================>]  83.84M  29.5MB/s    in 2.8s    

2020-08-10 00:39:43 (29.5 MB/s) - ‘/tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5’ saved [87910968/87910968]



cargamos el modelo de inception y elegimos la ultima layer

In [13]:
weights_path = "/tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5" # el path de donde estan ubicados los weights

pretrain_model = InceptionV3(input_shape=(150,150,3), #input de las imagenes
                             include_top = False, #elige si incluir la ultima neurona (output layer)
                             weights = None) #no incluir los weights del modelo

pretrain_model.load_weights(weights_path) #asignamos los pesos al modelo

for layer in pretrain_model.layers:
  layer.trainable = False #elegimos las neuronas como no entrenable

#pretrain_model.summary()

last_layer = pretrain_model.get_layer("mixed7") #elegimos la que queremos que sea la ultima neurona
last_layer_output = last_layer.output

In [14]:
from tensorflow.keras.optimizers import RMSprop

x = layers.Flatten()(last_layer_output) #asignamos como neurona consecuente a la ultima del inception, una flatten
x = layers.Dense(1024, activation='relu')(x) 
x = layers.Dropout(0.2)(x) #asignamos dropout a la anterior
x = layers.Dense  (1, activation='sigmoid')(x) #asignamos la ultima ultima

model = Model( pretrain_model.input, x) #creamos el modelo dandole un imput y una output

model.compile(optimizer = RMSprop(lr=0.0001), 
              loss = 'binary_crossentropy', 
              metrics = ['accuracy'])

In [15]:
!wget --no-check-certificate \ #descargamos los datos del
        https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip \
       -O /tmp/cats_and_dogs_filtered.zip


--2020-08-10 00:59:11--  https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.204.128, 64.233.189.128, 108.177.97.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.204.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 68606236 (65M) [application/zip]
Saving to: ‘/tmp/cats_and_dogs_filtered.zip’

/tmp/cats_and_dogs_ 100%[===================>]  65.43M  21.6MB/s    in 3.0s    

2020-08-10 00:59:15 (21.6 MB/s) - ‘/tmp/cats_and_dogs_filtered.zip’ saved [68606236/68606236]



In [18]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os 
import zipfile 

zip = zipfile.ZipFile("/tmp/cats_and_dogs_filtered.zip", "r")
zip.extractall("/tmp")
zip.close()

In [21]:
train_datagen = ImageDataGenerator( #creamos el data augmented para el train
    rescale = 1./255.,
    rotation_range = 40,
    width_shift_range = 0.2,
    height_shift_range = 0.2,
    shear_range = 0.2,
    zoom_range = 0.2,
    horizontal_flip = True)

test_datagen = ImageDataGenerator( rescale = 1./255. ) #creamos el data augmented para el test

train_generator = train_datagen.flow_from_directory( #leemos los datos y los label each one
    "/tmp/cats_and_dogs_filtered/train",
    batch_size = 20,
    class_mode = 'binary', 
    target_size = (150, 150)     
)

validation_generator = test_datagen.flow_from_directory( 
    "/tmp/cats_and_dogs_filtered/validation",
    batch_size = 20,
    class_mode = 'binary', 
    target_size = (150, 150)    
)


Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


In [ ]:
history = model.fit(
            train_generator,
            validation_data = validation_generator,
            steps_per_epoch = 100,
            epochs = 20,
            validation_steps = 50,
            verbose = 2)

In [ ]:
import matplotlib.pyplot as plt
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'r', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend(loc=0)
plt.figure()


plt.show()